<a href="https://colab.research.google.com/github/eunnjji/MultiCampus-EmpAcademy/blob/main/project2/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6조

- 팀명: SYH : super youth hero
- 주제: 기계 시설물 분야 AI 학습용 데이터를 활용하여 기계 시설물의 고장 여부 예측 모델 개발
- 여수진,성은지,홍승표  
- [참고](https://www.aidatahackathon.com/)

## 절차
1. 문제 분석  
 1.1 iris 데이터셋으로 회귀 연습
2. 데이터분석
3. 데이터전처리
4. 트레인, 검증 set 분리
5. 모델 생성
 

In [2]:
# libarary import
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn as skl

# seed setting
np.random.seed(42)
tf.random.set_seed(42)

BASE_DIR = "/content/drive/MyDrive/"
DATA_DIR = '/content/drive/MyDrive/project/daejeon_data'

## 데이터 형태
csv 파일  
,(콤마)로 분류된 시계열 데이터  
+ current Header  
  - 파일명, 에러 아이디, RPM, 정격전류,
  - 사용전류, Sample Rate 값, R전류의 RMS (RRMS),
  - R 전류의 MEAN (RMEAN), R 전류의 MAX (RMAX),
  - S 전류의 RMS (SRMS), S 전류의 MEAN (SMEAN),
  - S 전류의 MAX (SMAX), T 전류의 RMS (TRMS),
  - T 전류의 MEAN (TMEAN), T 전류의 MAX (TMAX)
+ kimm Header
  - 파일명, Sample Rate 값, Label, Label_No
+ vibration Header
  - 파일명, Sample Rate, Sample Rate, Sample Rate

## 데이터 구조

op-csv : 모터 정상 가동 중 측정한 데이터  
no-csv : 모터가 해당 고장 레이블인 상태에 측정한 데이터

current(전류 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
kimm(유선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  
vibration(무선진동 데이터) - __11, 15, 18.5, 2.2, 3.7, 5.5, 7.5__ (단위: kw)  

|레이블|
|---|
|정상|
|회전체 불평형(unbalance)|
|베어링 불량(bearing)|
|벨트 느슨함(belt)|
|축정렬 불량(misalignment)|
<br>

전류의 경우 \[r,s,t전류 값] target(y)는 \[레이블종류] 특성 \[kw]  
유선진동의 경우 \[] target(y)는 \[레이블종류] 특성 \[kw]  
무선진동의 경우 \[] target(y)는 \[레이블종류] 특성 \[kw]  

target을 원-핫-벡터로


# 1. 문제 분석

지도 학습 문제, 회귀 문제,   
다중 레이블 분류 ( 소프트맥스 회귀 )  
다층 퍼셉트론 이용(예정)
시계열 데이터 -> RNN ?


# 1.1. Iris Dataset으로 회귀 연습

In [3]:
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
# p197,4장 
type(iris['data']) # numpy.ndarray
iris['data'].shape # (150, 4) 
X = iris['data'][:,3:] 
y = (iris['target']==2).astype(np.int)

In [5]:
from sklearn.linear_model import LogisticRegression
# 로지스틱회귀, 이진분류
log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
log_reg.predict([[1.7],[1.5]])

array([1, 0])

In [7]:
#소프트맥스회귀, 다중분류
X = iris["data"][:, (2, 3)]  # petal length, petal width 꽃잎 길이, 너비
y = iris["target"]
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
softmax_reg.predict([[5, 2]]) # 꽃잎의 길이가 5, 꽃잎 너비가 2cm인 꽃 -> 클래스 2라고 예측

array([2])

In [9]:
#서포트벡터머신 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64) # feature scailing 필요
svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", LinearSVC(C=1,loss='hinge'))
])
svm_clf.fit(X,y)
svm_clf.predict([[5, 2]])

array([1.])

In [10]:
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier
svm_clf2 = Pipeline([
                    ("scaler", StandardScaler()),
                    ("SVC", SVC(kernel='hinge',C=1))
])

In [ ]:
svm_clf3 = Pipeline([
                    ("scaler", StandardScaler()),
                    # ("sgd_clf", SGDClassifier(loss='hinge',alpha= )
])

In [ ]:
#결정트리


In [ ]:
#랜덤포레스트

In [43]:
#투표기반 분류

In [44]:
#앙상블

# 2. 데이터 분석


In [ ]:
# !unzip -uq "/content/drive/MyDrive/경진대회_data.zip" -d "압축 푼 파일 저장할 경로"

,로 구분된 csv 파일 형태로 데이터 가져오기

In [ ]:
%cd '/content/drive/MyDrive'
os.getcwd()

'/content/drive/My Drive'

In [ ]:
classSet = ("bearing", "belt", "misalignment", "unbalance")

def DatadirLoad(dirname):
    csv_op_dict = {}
    csv_no_dict = {}
    # 'label': str? list[float]?
    # 'data' : [ { }  ]

    dirpath = os.path.join(DATA_DIR,dirname)
    if os.path.isdir(dirpath):
        for d in os.listdir(dirpath):
            d_path = dirpath+'/'+d
            if os.path.isdir(d_path):
                Error_Class_Lable,kw_Name,*_ = d.split('_')
                print(Error_Class_Lable,kw_Name)
                data_dir_list = [ x for x in os.listdir(d_path) if len(x)<2]
                data_dir_list.sort()
                print(data_dir_list)
                if len(data_dir_list) > 0:
                    for d2 in data_dir_list:
                        d2_path = os.path.join(d_path,d2)
                        # op-csv file
                        d2_op_path = os.path.join(d2_path,'4.op-csv')
                        d2_op_data = [x for x in os.listdir(d2_op_path) if x.endswith('.csv')]
                        csv_op_dict[Error_Class_Lable].append(list((d2_op_data[0],kw_Name)))
                        op_csv_path = os.path.join(d2_op_path,d2_op_data[0])
                        if os.path.isfile(op_csv_path):
                            # csv_op_list.append(pd.read_csv(op_csv_path,sep=',').) # error
                            print('open save')

                        # no-csv file
                        d2_no_path = os.path.join(d2_path,'5.no-csv')
                        d2_no_data = [x for x in os.listdir(d2_no_path) if x.endswith('.csv')]
                        csv_no_dict[Error_Class_Lable].append(list((d2_no_data,kw_Name)))
                

    return csv_op_dict, csv_no_dict

In [ ]:
current_op_csv,current_no_csv = DataLoad("current(전류)")
for k,v in current_op_csv.items():
    print(k)
    print(len(v))
    # for vi in v:
    #     print(vi)
for k,v in current_no_csv.items():
    print(k)
    print(len(v))  

In [ ]:
kimm_op_csv, kimm_no_csv = DataLoad("kimm(유선진동)")
for k,v in kimm_op_csv.items():
    print(k)
    print(len(v))
    # for vi in v:
        # print(vi)
for k,v in kimm_no_csv.items():
    print(k)
    print(len(v))  

In [ ]:
vibration_op_csv, vibration_no_csv = DataLoad("vibration(무선진동)")
for k,v in vibration_op_csv.items():
    print(k)
    print(len(v))
    # for vi in v:
        # print(vi)
for k,v in vibration_no_csv.items():
    print(k)
    print(len(v))  